# Working with Zarr

## Intro

`titiler.xarray` is a submodule designed specifically for working with multidimensional dataset. With version `0.25.0`, we've introduced a default application with only support for Zarr dataset.

In [ ]:
# setup
import httpx
import json
from IPython.display import Image

# Developmentseed Demo endpoint. Please be kind. Ref: https://github.com/developmentseed/titiler/discussions/1223
# titiler_endpoint = "https://xarray.titiler.xyz"

# Or launch your own local instance with:
# uv run --group server uvicorn titiler.xarray.main:app --host 127.0.0.1 --port 8080 --reload
titiler_endpoint = "http://127.0.0.1:8080"

zarr_url = "https://nasa-power.s3.us-west-2.amazonaws.com/syn1deg/temporal/power_syn1deg_monthly_temporal_lst.zarr"

## Dataset Metadata

The `/dataset/dict` endpoint returns general metadata about the Zarr Dataset

Endpoint: `/dataset/dict`

QueryParams:
- **url**: Zarr store URL

In [ ]:
r = httpx.get(
    f"{titiler_endpoint}/dataset/dict",
    params={
        "url": zarr_url,
    },
).json()

print(json.dumps(r, indent=2))

#### List of available variables

Endpoint: `/dataset/keys`

QueryParams:
- **url**: Zarr store URL

In [ ]:
r = httpx.get(
    f"{titiler_endpoint}/dataset/keys",
    params={
        "url": zarr_url,
    },
).json()

print(json.dumps(r, indent=2))

## Variable Info

We can use `/info` endpoint to get more `Geo` information about a specific variable.

QueryParams:
- **url**: Zarr store URL
- **variable**: Variable's name (e.g `AIRMASS`, found in `/dataset/keys` response)

In [ ]:
r = httpx.get(
    f"{titiler_endpoint}/info",
    params={"url": zarr_url, "variable": "AIRMASS"},
).json()

print(json.dumps(r, indent=2))

Or as a GeoJSON feature

In [ ]:
r = httpx.get(
    f"{titiler_endpoint}/info.geojson",
    params={"url": zarr_url, "variable": "AIRMASS"},
).json()

print(json.dumps(r, indent=2))

### Knowledge

Looking at the `info` response we can see that the `AIRMASS` variable has `348` (count) bands, each one corresponding to as specific `TIME` (day).

We can also see that the data is stored as `float32` which mean that we will have to apply linear rescaling in order to get output image as PNG/JPEG.

The `min/max` values are also indicated with `valid_max=31.73` and `valid_min=1.0`.


### Dimension Reduction

We cannot visualize all the `bands` at once, so we need to perform dimension reduction to go from array in shape (348, 360, 180) to a 1b (1, 360, 180) or 3b (3, 360, 180) image. 

To do it, we have two methods whitin `titiler.xarray`:
- using `bidx=`: same as for COG we can select a band index
- using `sel={dimension}=value`: which will be using xarray `.sel` method

In [ ]:
r = httpx.get(
    f"{titiler_endpoint}/bbox/-180,-90,180,90.png",
    params=(
        ("url", zarr_url),
        ("variable", "AIRMASS"),
        # Select 1 specific band
        ("bidx", 50),
        ("rescale", "1,20"),
        ("colormap_name", "viridis"),
    ),
    timeout=10,
)

Image(r.content)

In [ ]:
r = httpx.get(
    f"{titiler_endpoint}/bbox/-180,-90,180,90.png",
    params=(
        ("url", zarr_url),
        ("variable", "AIRMASS"),
        # Select 1 specific time slices
        ("sel", "time=2003-06-30"),
        ("rescale", "1,20"),
        ("colormap_name", "viridis"),
    ),
    timeout=10,
)

Image(r.content)

In [ ]:
r = httpx.get(
    f"{titiler_endpoint}/bbox/-180,-90,180,90.png",
    params=(
        ("url", zarr_url),
        ("variable", "AIRMASS"),
        # Select 3 specific time slices to create a 3 band image
        ("sel", "time=2003-06-30"),
        ("sel", "time=2004-06-30"),
        ("sel", "time=2005-06-30"),
        ("rescale", "1,10"),
    ),
    timeout=10,
)

Image(r.content)